In [ ]:
import numpy as np
import pandas as pd
import os
import keras
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Dropout, BatchNormalization
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras import regularizers
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
import seaborn as sns
import random
from keras.callbacks import ModelCheckpoint

In [ ]:
data_dir = '/content/drive/MyDrive/data/consonants'
CATEGORIES = ['1','4','6','8','10','11','12','13','16','18','19','20',
              '21','22','23','25','27','28','29','31','32','33','34','35','36']

In [ ]:
def load_unique(DIR):
    images_for_plot = []
    labels_for_plot = []
    size_img = 64,64
    for category in CATEGORIES:
        path = os.path.join(DIR,category)
        class_num = category
        print(category, end = ' | ')
        for img in os.listdir(path):
            image = cv2.imread(os.path.join(path,img))
            final_img = cv2.resize(image, size_img)
            final_img = cv2.cvtColor(final_img, cv2.COLOR_BGR2GRAY)
            images_for_plot.append(final_img)
            labels_for_plot.append(class_num)
            break
    return images_for_plot, labels_for_plot

images_for_plot, labels_for_plot = load_unique(data_dir)
print("unique_labels = ", labels_for_plot)

fig = plt.figure(figsize = (15,15))
def plot_images(fig, image, label, row, col, index):
    fig.add_subplot(row, col, index)
    plt.axis('off')
    plt.imshow(image, cmap = 'gray')
    plt.title(label)
    return

image_index = 0
row = 5
col = 6
for i in range(1,(row*col)):
    if i > 25:
      break
    plot_images(fig, images_for_plot[image_index], labels_for_plot[image_index], row, col, i)
    image_index = image_index + 1
plt.show()

In [ ]:
def load_data_train(DIR):
    train_data = []
    size = 32,32
    print("LOADING DATA FROM : ",end = "")
    for category in CATEGORIES:
        path = os.path.join(DIR,category)
        class_num=CATEGORIES.index(category)
        print(category, end = ' | ')
        for img in os.listdir(path):
            img_array=cv2.imread(os.path.join(path,img))
            try:
                new_array = cv2.resize(img_array,size)
                final_img = cv2.cvtColor(new_array, cv2.COLOR_BGR2GRAY)
                train_data.append([final_img,class_num])
            except:
                print(os.path.join(path,img))
    random.shuffle(train_data)
    X=[]
    Y=[]
    for features, label in train_data:
        X.append(features)
        Y.append(label)
    X = np.array(X).reshape(-1,32,32,1)
    Y = np.array(Y)
    X = X.astype('float32')/255.0
    
    Y = keras.utils.to_categorical(Y,25)
    
    X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2)
    
    print()
    print('Loaded', len(X_train),'images for training,','Train data shape =',X_train.shape)
    print('Loaded', len(X_test),'images for testing','Test data shape =',X_test.shape)
    
    return X_train, X_test, Y_train, Y_test

In [ ]:
X_train, X_test, Y_train, Y_test = load_data_train(data_dir)

In [ ]:
def create_model():
    model = Sequential()
    model.add(Conv2D(32, kernel_size = [3,3], activation = 'relu', input_shape = (32,32,1)))
    model.add(Conv2D(64, kernel_size = [3,3], activation = 'relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="same"))
    
    model.add(Conv2D(64, kernel_size = [3,3], activation = 'relu'))
    model.add(Conv2D(64, kernel_size = [3,3], activation = 'relu'))
    model.add(MaxPool2D(pool_size=(2, 2), strides=(2, 2), padding="same"))

    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(128, activation = 'relu'))
    model.add(Dense(64, activation = 'relu'))
    model.add(Dense(25, activation = 'softmax'))
    model.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ["accuracy"])
    print("MODEL CREATED")
    model.summary()
    return model

def fit_model():
    checkpointer = ModelCheckpoint(filepath='model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
    model_hist = model.fit(X_train, Y_train, batch_size = 32, epochs = 3,
          validation_split = 0.2, callbacks = [checkpointer], 
          verbose=1, shuffle=True)
    return model_hist 

In [ ]:
model = create_model()
model_hist = fit_model()

MODEL CREATED
Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_44 (Conv2D)           (None, 30, 30, 32)        320       
_________________________________________________________________
conv2d_45 (Conv2D)           (None, 28, 28, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 14, 14, 64)        0         
_________________________________________________________________
conv2d_46 (Conv2D)           (None, 12, 12, 64)        36928     
_________________________________________________________________
conv2d_47 (Conv2D)           (None, 10, 10, 128)       73856     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 5, 5, 128)         0         
_________________________________________________________________
dropout_14 (Dropout)         (None, 5, 

In [ ]:
model.load_weights('model.weights.best.hdf5')

In [ ]:
evaluate_metrics = model.evaluate(X_test, Y_test)
print("\nEvaluation Accuracy = ", "{:.2f}%".format(evaluate_metrics[1]*100),"\nEvaluation loss = " ,"{:.6f}".format(evaluate_metrics[0]))

313/313 [==============================] - 12s 38ms/step - loss: 0.0962 - accuracy: 0.9716

Evaluation Accuracy =  97.16% 
Evaluation loss =  0.096222
